In [1]:
#PRECISO PEGAR OS DADOS DA TABELA PRINCIPAL, ESSES DADOS AI SAO DO MODAL.
#DPS DE FAZER ISSO, VOU COLOCAR OS DADOS DA TAB PRINCIAPL NO BANCO E DPS DO MODAL
# o banco vai ter 2 tabelas, uma com os dados principais e outra com os dados do mal referente a esse item principal

In [8]:
from selenium import webdriver
from core import MySQLcompatible
import requests
from bs4 import BeautifulSoup
import re
import utils
import json
import time

In [2]:
tabelas = {}

EXPLICAÇÃO - tabela **jogos_uni**

id-> id do jogo

titulo-> time1 x time2 (onde vc colocou partida)

data -> data/hora 

slugLiga-> é o vc colocou nome_campeonato dessa forma (inglaterra-partidas-reservas)sem espaço e sem acento tudo minusculo.Tipo uma url amigavel

pais-> é o pais (a primeira parte do nome_campeonato)

liga -> é o campeonato (é a segunda parte do nome_campeonato) 

status-> valor fixo 1

posicao-> valor fixo 1 

In [3]:
tabelas['jogos_uni'] = (
"CREATE TABLE `jogos_uni` ("
"  `id` int(11) NOT NULL,"
"  `titulo` varchar(250) NOT NULL,"
"  `data` datetime NOT NULL,"
"  `slugLiga` varchar(120) DEFAULT NULL,"
"  `pais` varchar(100) DEFAULT NULL,"
"  `liga` varchar(100) DEFAULT NULL,"
"  `status` int(11) NOT NULL,"
"  `posicao` int(11) NOT NULL"
") ENGINE=MyISAM DEFAULT CHARSET=latin1")

EXPLICAÇÃO - tabela **modal_uni**

id-> auto increment

jogo_id-> id do jogo

odd_id -> id da odd

cat_id-> valor fixo 1

categoria -> [tipoModalidae] 

propriedade -> [modalidade]

valor->[odd]

id_modal->valor fixo 1

status -> valor fixo 1

In [4]:
tabelas['modal_uni'] = (
"CREATE TABLE IF NOT EXISTS `modal_uni` ("
"  `id` int(11) NOT NULL,"
"  `jogo_id` int(11) NOT NULL,"
"  `odd_id` int(11) NOT NULL,"
"  `cat_id` int(11) NOT NULL,"
"  `categoria` varchar(250) NOT NULL,"
"  `propriedade` varchar(250) NOT NULL,"
"  `valor` decimal(8,2) NOT NULL DEFAULT '0.00',"
"  `status` int(11) NOT NULL"
") ENGINE=MyISAM DEFAULT CHARSET=latin1;")


In [5]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)
browser.get("https://bets93.net/")
browser.find_element_by_xpath("//div[@class='lateral']/div/ul").click() #clica no botao "todos"

In [26]:
pattern_campeonato = re.compile(r"c_visivel")
pattern_jogo = re.compile(r"j_visivel_")

In [27]:
soup = BeautifulSoup(browser.page_source, "html.parser")

In [30]:
tabela_jogos = soup.find(class_="jogos")
jogos = tabela_jogos.findAll("div")

In [ ]:
soup

In [144]:
def concat(lista):
    x = ''
    for i in lista:
        x += i
    return x

In [190]:
def concat_titulo(lista):
    string = ''
    for i in range(len(lista)):
        string += lista[i]+' '
    return string

In [191]:
concat_titulo(['Atletico', 'Mineiro', 'x', 'Vasco', 'da', 'Gama'])

'Atletico Mineiro x Vasco da Gama '

In [219]:
for jogo in jogos:
    attr = jogo.get("id")
    if attr: pass
    else: continue
    is_camp = pattern_campeonato.match(attr)
    is_jogo = pattern_jogo.match(attr)
    if is_camp:        
        search = jogo.find(class_="camp")
        if search:
            try:
                liga = search.text.split()[1:2][0]+' '+search.text.split()[2:][0]
            except Exception:
                liga = search.text.split()[1:2][0]
            campeonato = re.sub(r'(^-)', '',search.text.replace(' ','-').lower())
        else: continue
    elif is_jogo:
        id_html = jogo.get("id")
        if id_html: 
            id_jogo = id_html.split('_')[-1]
            id_botao = "jogo_"+id_jogo+"_outros"
        else: continue
        times_fundojogos = jogo.find(class_="times fundojogos")
        if times_fundojogos:
            partida = concat_titulo(times_fundojogos.text.split()[:-3])
            data_hora = times_fundojogos.find(class_="datahora").text
        else: continue
    dados = {'id':id_jogo, 'titulo':partida, 'slugLiga': campeonato, 'pais': campeonato.split('-')[0],
             'liga':liga, 'status':1, 'posicao':1}
    print(json.dumps(dados, indent=4))

{
    "id": "83178777",
    "titulo": "St Lucia Women x US Virgin Islands Women ",
    "slugLiga": "brazil-serie-a",
    "pais": "brazil",
    "liga": "Serie A",
    "status": 1,
    "posicao": 1
}
{
    "id": "82938640",
    "titulo": "Atletico Mineiro x Vasco da Gama ",
    "slugLiga": "brazil-serie-a",
    "pais": "brazil",
    "liga": "Serie A",
    "status": 1,
    "posicao": 1
}
{
    "id": "82938640",
    "titulo": "Atletico Mineiro x Vasco da Gama ",
    "slugLiga": "brazil-serie-a",
    "pais": "brazil",
    "liga": "Serie A",
    "status": 1,
    "posicao": 1
}
{
    "id": "82938640",
    "titulo": "Atletico Mineiro x Vasco da Gama ",
    "slugLiga": "brazil-serie-a",
    "pais": "brazil",
    "liga": "Serie A",
    "status": 1,
    "posicao": 1
}
{
    "id": "82938640",
    "titulo": "Atletico Mineiro x Vasco da Gama ",
    "slugLiga": "brazil-serie-a",
    "pais": "brazil",
    "liga": "Serie A",
    "status": 1,
    "posicao": 1
}
{
    "id": "82938640",
    "titulo": "Atl